In [0]:
# This notebook is meant to run batch inference on the top of new iris samples

In [0]:
%pip install mlflow=='3.4.0'
dbutils.library.restartPython()

In [0]:
from sklearn import datasets
from mlflow.pyfunc import load_model
import pandas as pd
import mlflow
from datetime import datetime

In [0]:
dbutils.widgets.text("catalog_name", "pedroz_e2edata_dev")
catalog_name = dbutils.widgets.get("catalog_name")

catalog_name = "testes"

In [0]:
model_name = f'{catalog_name}.mlops.iris_model'

In [0]:
# Pull the dataset for running the inference
iris_samples = datasets.load_iris(as_frame=True)
df_samples = pd.DataFrame(data = iris_samples['data'], columns = iris_samples['feature_names'])
df_samples.columns = df_samples.columns.str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
df_samples.head()

In [0]:
model_uri = f"models:/{model_name}@challenger"
model = load_model(model_uri)

In [0]:
predictions = model.predict(df_samples)
df_samples['prediction'] = predictions

df_samples.head()

In [0]:
df_samples['actual_label'] = iris_samples['target']
df_samples.head()

In [0]:
# Adding the model_id and prediction_timestamp columns to the dataframe - 
# these are required if, in the future, you want to use Lakehouse Monitoring to track the performance of the model
mlflow_client = mlflow.tracking.MlflowClient()
model_version = mlflow_client.get_model_version_by_alias(model_name, "challenger").version

df_samples['prediction_timestamp'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
df_samples['model_id'] = model_version

display(df_samples)

In [0]:
df_spark = spark.createDataFrame(df_samples)

In [0]:
try:
    display(spark.table(f"{catalog_name}.mlops.iris_data").limit(5))
    table_exists = True
except:
    table_exists = False

In [0]:
if table_exists: # append
    df_spark.write.mode("append").saveAsTable(f"{catalog_name}.mlops.iris_inferences")
else: # create table from scratch
    df_spark.write.mode("overwrite").saveAsTable(f"{catalog_name}.mlops.iris_inferences")

In [0]:
display(spark.sql(f"SELECT * FROM {catalog_name}.mlops.iris_inferences LIMIT 5"))

In [0]:
# Enabling the Change Data Feed is a recommended practice for Inference Monitoring using Lakehouse Monitoring
# When CDF is enabled, only newly appended data is processed. 
spark.sql(f"ALTER TABLE {catalog_name}.mlops.iris_inferences SET TBLPROPERTIES (delta.enableChangeDataFeed = true)")